# Technical Challenge - Early Bark Beetle Detection Demo

Ce notebook démontre l'utilisation du pipeline de détection précoce des perturbations forestières avec un focus sur les épidémies de scolytes.

## Objectifs du Notebook

1. **Exécution d'inférence sur la ROI**
2. **Visualisation des prédictions de perturbation**
3. **Estimation du timing des perturbations avec ruptures**
4. **Utilisation des données ERA5 pour séparer vent vs scolytes**
5. **Évaluation de la précision/rappel et délai de détection**


In [ ]:
# Import des bibliothèques nécessaires
import sys
import os
sys.path.append('../../')

import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration du style des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Bibliothèques importées avec succès")


In [ ]:
# Chargement de la configuration
config_path = Path('../../configs/config_test.yaml')
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print("📋 Configuration chargée:")
print(f"  - ROI: {config['roi']['geojson_path']}")
print(f"  - Période: {config['date_range']['start']} à {config['date_range']['end']}")
print(f"  - Bandes Sentinel-2: {config['sentinel2']['bands']}")
print(f"  - Seuils de vent: {config['wind_analysis']['wind_thresholds']}")


## 1. Exécution d'Inférence sur la ROI

Exécutons le pipeline complet pour analyser la région d'intérêt.


In [ ]:
# Chargement des résultats de détection
results_dir = Path('../../results')
detections_dir = results_dir / 'detections'

print("📁 Répertoires de résultats:")
for subdir in results_dir.iterdir():
    if subdir.is_dir():
        print(f"  - {subdir.name}: {len(list(subdir.rglob('*')))} fichiers")

# Vérifier les masques de détection
if detections_dir.exists():
    print(f"\n🔍 Masques de détection disponibles:")
    for mask_file in detections_dir.glob('*.tif'):
        print(f"  - {mask_file.name}")
else:
    print("⚠️ Aucun masque de détection trouvé")


In [ ]:
# Visualisation des masques de détection
import rasterio
import matplotlib.patches as patches

def plot_detection_masks(detections_dir):
    """Visualiser les masques de détection"""
    if not detections_dir.exists():
        print("⚠️ Aucun masque de détection à visualiser")
        return
    
    # Trouver les fichiers de masques
    mask_files = list(detections_dir.glob('*.tif'))
    if not mask_files:
        print("⚠️ Aucun fichier .tif trouvé")
        return
    
    # Créer la figure
    n_masks = len(mask_files)
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    for i, mask_file in enumerate(mask_files[:4]):  # Limiter à 4 masques
        try:
            with rasterio.open(mask_file) as src:
                data = src.read(1)
                transform = src.transform
                
                # Créer l'image
                im = axes[i].imshow(data, cmap='viridis', aspect='equal')
                axes[i].set_title(f'{mask_file.stem}', fontsize=12, fontweight='bold')
                axes[i].set_xlabel('Pixels X')
                axes[i].set_ylabel('Pixels Y')
                
                # Ajouter la colorbar
                plt.colorbar(im, ax=axes[i], shrink=0.8)
                
        except Exception as e:
            axes[i].text(0.5, 0.5, f'Erreur:\n{str(e)[:50]}...', 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].set_title(f'{mask_file.stem} (Erreur)')
    
    # Masquer les axes inutilisés
    for i in range(len(mask_files), 4):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('Masques de Détection de Changements', fontsize=16, y=1.02)
    plt.show()

# Exécuter la visualisation
plot_detection_masks(detections_dir)


## 3. Estimation du Timing des Perturbations avec Ruptures

Analyse de la précision temporelle de la détection des changements.


In [ ]:
# Analyse du timing des perturbations
def analyze_timing_precision(detections_dir):
    """Analyser la précision temporelle des détections"""
    
    # Charger les données de timing si disponibles
    timing_file = detections_dir / 'ruptures_first_change.tif'
    if not timing_file.exists():
        print("⚠️ Fichier de timing non trouvé")
        return
    
    try:
        with rasterio.open(timing_file) as src:
            timing_data = src.read(1)
            
        # Statistiques de timing
        valid_times = timing_data[timing_data > 0]
        
        if len(valid_times) == 0:
            print("⚠️ Aucune détection de timing valide")
            return
        
        print("📅 Analyse de la précision temporelle:")
        print(f"  - Nombre de pixels avec changements: {len(valid_times)}")
        print(f"  - Timing moyen: {np.mean(valid_times):.2f} jours")
        print(f"  - Timing médian: {np.median(valid_times):.2f} jours")
        print(f"  - Écart-type: {np.std(valid_times):.2f} jours")
        print(f"  - Timing min: {np.min(valid_times):.2f} jours")
        print(f"  - Timing max: {np.max(valid_times):.2f} jours")
        
        # Visualisation de la distribution
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.hist(valid_times, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        plt.xlabel('Timing du changement (jours)')
        plt.ylabel('Fréquence')
        plt.title('Distribution du Timing des Changements')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.boxplot(valid_times, patch_artist=True, 
                   boxprops=dict(facecolor='lightgreen', alpha=0.7))
        plt.ylabel('Timing du changement (jours)')
        plt.title('Boxplot du Timing des Changements')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse du timing: {e}")

# Exécuter l'analyse
analyze_timing_precision(detections_dir)


# Technical Test - Early Bark Beetle Detection with Sentinel Data

## Démonstration du Pipeline Complet

Ce notebook démontre l'utilisation du pipeline modulaire pour la détection précoce des perturbations forestières avec focus sur les épidémies de scolytes.
